In [ ]:


select tags from tb_algo where locate("机器",tags);

In [10]:
#%%file Arthur_serviceTags_dao.py
from Arthur.core.entities.arthur_web_cfg import *
from Arthur.core.entities.base.bs_util import *
from Arthur.core.entities.base.bs_time import get_cur_day,get_cur_time
from Arthut.core.entities.dao.Arthur_serviceTags_dao import ArthurServTagsDao
#2019-01-30 add
class ArthurServTags:
    
    #用户的tag service list
    @staticmethod
    def tag_service_list(status='normal', search=None):
        dataBase = DataBase()
        bRet, sRet = ArthurServTagsDao.query_node_list_by_tag(search='机器')
        if not bRet:
            return False, sRet

        return True, sRet

if __name__ == "__main__":

    #ArthurAlgoStarDao.insert_node(algouid, uid,algoname,star_status,thumb_status)
    ArthurServTagsDao.query_node_list_by_tag(search='机器')


Writing Arthur_serviceTags_dao.py


In [ ]:
# -*- coding: UTF-8 -*-
# author: leepand
# time: 2019-01-08
# desc: Algo 数据展示(my/other/zoo)

#update 2019-01-25

import datetime
from operator import itemgetter 
from Arthur.core.entities.bean.Arthur_service import ArthurService
from Arthur.core.entities.dao.Arthur_AlgoStar_dao import ArthurAlgoStarDao
from Arthut.core.entities.dao.Arthur_serviceTags_dao import ArthurServTagsDao

class ArthurAlgoList:

    @staticmethod
    def _get_algolist_byuid(uId):
        #增加收藏的算法服务
        bRet,uid_algouid_list = ArthurAlgoStarDao.query_algouid_by_uidstar(uId)
        my_star_algo_list_all=[]
        my_star_algo_list=[]
        my_star_algo_list_dict=None
        if len(uid_algouid_list)>0:
            for algouid_algoname_dict in uid_algouid_list:
                algouid=algouid_algoname_dict['algouid']
                algoname = algouid_algoname_dict['algoname']
                #过滤掉自己收藏自己的算法，避免重复
                if int(algouid)-int(uId)==0:
                    continue
                bRet, my_star_algo_list_dict = ArthurService.service_user_list_filter(algouid,algoname,page=1,length=1000, status='normal', search=None)
                #print 'my_star_algo_list_dict',my_star_algo_list_dict
                if not bRet: my_star_algo_list = []
                if my_star_algo_list_dict is not None and "service_list" in my_star_algo_list_dict:
                    my_star_algo_list= my_star_algo_list_dict['service_list']
                else:
                    my_star_algo_list=[]
                my_star_algo_list_all.extend(my_star_algo_list)
    
        bRet, my_algo_list_dict = ArthurService.service_user_list(uId,page=1,length=1000, status='normal', search=None)
        if not bRet: my_algo_list = []
        if my_algo_list_dict is not None and "service_list" in my_algo_list_dict:
            my_algo_list= my_algo_list_dict['service_list']
        else:
            my_algo_list=[]
        my_algo_list.extend(my_star_algo_list_all)
        my_algo_list_new=my_algo_list
        return  {
            "algo_list": my_algo_list_new
        }
    
    @staticmethod
    def _get_algolist_zoo():
        bRet, algo_list_zoo = ArthurService.service_zoo_list(page=1,length=1000, status='normal', search=None)
        if not bRet: algo_list_zoo = []
        if  algo_list_zoo is not None and "service_list" in algo_list_zoo:
            serviceList=algo_list_zoo['service_list']
            #serviceList_bystar = sorted(serviceList,key = itemgetter('star_cnt'),reverse = True)
            serviceList_bytime = sorted(serviceList,key = itemgetter('insert_tm'),reverse = True)

        else:
            serviceList=[]
            serviceList_bystar=[]
            serviceList_bytime=[]
        return {
                "algo_list": serviceList,#serviceList
                "algo_list_bytime":serviceList_bytime}



    @staticmethod
    def data_show(uId,query_type):
        if query_type=='zoo':
            datas = {
                "dt_algo_list":ArthurAlgoList._get_algolist_zoo()
            }
        else:
            datas = {
                "dt_algo_list":ArthurAlgoList._get_algolist_byuid(uId)
            }
        return True, datas
    @staticmethod
    def algo_list_forautocomplete():
        bRet, sRet = ArthurAlgoList.data_show(1,'zoo')
        new_serv_list=[]
        try:
            for _inter in sRet['dt_algo_list']['algo_list']:
                new_serv_list.append(_inter['algoname'])
        except:
            pass
        return True,new_serv_list

if __name__ == "__main__":

    bRet, sRet = ArthurAlgoList.data_show(1,'zoo')

    print '>>>> ', sRet

    #print HjsIndex._get_expire_order()
